Now we try reducing the dimensinos with autoenconders

In [4]:
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.cluster import KMeans

import pickle

import itertools
from tabulate import tabulate

import tensorflow as tf
import keras
from keras import layers
from keras import regularizers
from keras import initializers

In [5]:
#Same clustering function as before
#but we include the encoder variables, Xi, in output for extra insight 
def kmeans_classify(X, exclusions, nclasses):
    #Perform clustering
    kmeans = KMeans(n_clusters=nclasses, tol=1e-6).fit(X)
    #Get labels for all heroes
    indx = kmeans.labels_
    
    #Order clusters by size
    cls_size = np.unique(indx, return_counts=True)[1]
    sorted_indx = np.argsort(cls_size)

    l=[]
    for i in sorted_indx:
        ltemp = list(heroes[np.argwhere(indx==i)].flatten())
        ltemp.sort()
        l.append(ltemp)
    ltrans = list(map(list,itertools.zip_longest(*l,fillvalue=None)))
    print(tabulate(ltrans))


    header2 = np.delete(header,exclusions,0)
    stats2 = np.delete(stats,exclusions,1)
    stats3 = np.append(stats2,X*100,1)
    
    cls_avg = []
    cls_avg = [np.append('Metric - Class#',header2)]
    
    for i in np.arange(X.shape[1]):
        cls_avg = [np.append(cls_avg,'X%d'%i)]
    
    j=0
    for i in sorted_indx:
        avg = np.round(np.mean(stats3[np.argwhere(indx==i).flatten()],axis=0),1)
        std = np.std(stats3[np.argwhere(indx==i).flatten()],axis=0)
        cv = np.round(std/(avg+1e-6),2)
        avg = np.append('Avg. %d'%j,avg)
        std = np.append('CV. %d'%j,cv)
        j+=1

        cls_avg.append(avg)
        cls_avg.append(std)
    cls_avg = list(map(list,zip(*cls_avg)))
    print(tabulate(cls_avg))
    
#kmeans_classify(encoder.predict(X),exclusions,4)

In [480]:
blah1 = np.delete(stats,exclusions,1)
blah2 = Xenc
np.append(blah1,blah2,axis=1)[:,13]

array([0.64783496, 0.48512653, 0.39276606, 0.54037249, 0.41626233,
       0.52498758, 0.65004361, 0.52021652, 0.63182694, 0.4368248 ,
       0.62578982, 0.74833304, 0.51314652, 0.51577103, 0.52543354,
       0.54308546, 0.66859841, 0.76350713, 0.64563107, 0.45748296,
       0.70251721, 0.55985069, 0.53105825, 0.42984372, 0.47271848,
       0.50946397, 0.64957511, 0.62811458, 0.51486206, 0.51108694,
       0.43201044, 0.51565015, 0.64084631, 0.56530499, 0.57579392,
       0.51863509, 0.61095858, 0.46296731, 0.74589807, 0.78224409,
       0.59886867, 0.37543839, 0.49414995, 0.45324948, 0.55935436,
       0.70910442, 0.56982243, 0.63696671, 0.58043176, 0.47287184,
       0.68670553, 0.49759963, 0.50784516, 0.58852905, 0.46211126,
       0.65114999, 0.43076625, 0.32511395, 0.31849894, 0.56625134,
       0.44002292, 0.36695841, 0.41803375, 0.60367262, 0.39601874,
       0.54169434, 0.27375081, 0.1602159 , 0.28683397, 0.58295381,
       0.36884072, 0.19047911, 0.71850365, 0.28988224, 0.77762

In [446]:
blah3 = np.delete(header,exclusions,0)
blah4 = np.arange(Xenc.shape[1])

blah5 = [np.append('Metric - Class#',blah3)]
np.append(blah5,blah4)

array(['Metric - Class#', 'Takedowns', 'Kills', 'Deaths', 'Hero Dmg',
       'Siege Dmg', 'Healing', 'Self Heal', 'Dmg Taken', 'XP', 'DPS',
       'Range', 'HP', '0', '1'], dtype='<U21')

In [474]:
np.mean(Xenc[:,1])

0.46885055

In [432]:
Xenc=encoder.predict(X)

In [6]:
#Load data
d = np.load('hots_stats_ext.npz')
header = d['header'][1:]
heroes = d['heroes']
stats = d['stats']

In [ ]:
header

In [7]:
#Normalize data
normalizer = preprocessing.Normalizer()
norm_stats = normalizer.transform(stats)  

In [8]:
#Remove Games, Win rate, Avg. game duration (superfluous) and T/D (redundant)
#For now also exclude internal stats: DPS, Range, HP
exclusions = [0,1,2,3]
X = np.delete(norm_stats,exclusions,1)

In [33]:
#We'll test the encoder manually here then run it through a script
#We're getting 0 columns in some instances
#we tried changing seeds and adding dropout, no difference
#Since run time is short, we'll brute force and run many trials, pick the best result

input_dim = X.shape[1]
encoding_dim = 2
print('Input dim: ',input_dim)
print('Encoding dim: ',encoding_dim)

Xin = keras.Input(shape=(input_dim,))

Xin = layers.Dropout(0.1)(Xin)

encoded = layers.Dense(encoding_dim,
                       activation='relu')(Xin)
decoded = layers.Dense(input_dim, 
                       activation='relu')(encoded)

autoencoder = keras.Model(Xin, decoded)

encoder = keras.Model(Xin,encoded)

autoencoder.compile(optimizer='adam',loss='mean_squared_error')

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

history = autoencoder.fit(X,X,
                epochs = 1000,
                batch_size=90,
                verbose=0,
                callbacks=callback);

#print(autoencoder.evaluate(x=X,y=X, verbose='1'))
print(history.history['loss'][-1])
print(len(history.history['loss']))
encoder.predict(X)[:10]

Input dim:  12
Encoding dim:  2
0.010190555825829506
1000


array([[0.        , 1.056745  ],
       [0.        , 0.641669  ],
       [0.        , 0.431709  ],
       [0.        , 0.8127952 ],
       [0.        , 0.43308413],
       [0.        , 0.7514824 ],
       [0.        , 0.89771414],
       [0.        , 0.7885611 ],
       [0.        , 0.9112069 ],
       [0.        , 0.5759032 ]], dtype=float32)

In [16]:
history.history['loss'][-1]

0.02565278671681881

In [9]:
#Experiment 0 with auto1
#20 trials x 1000 epochs
#Picked the lowest loss
data = pickle.load(open('auto1_exp0.pickle','rb'))

In [71]:
indx = 5 #Data set index
numclass = 5

print('Dimensions =',data[indx][0])
print('Loss =',data[indx][1])
kmeans_classify(data[indx][2],exclusions,numclass)

#We compare clustering with exp_0 (w/o dim-red)
#Dim:Comments
#2,3:healers in tank sometimes
#They do have similar takedown because they partake in hero fights
#But other stats are very different
#4: some migration between damage classes. Clustering unstable
#5: Unstable. Tank class might draw melee assasins. Sonya left tank; good
#6: Stable. Tanks, healers look good. Some damage dealers are surprising

Dimensions = 6
Loss = 0.00027452537324279547
---------  -----------  ---------  ----------------  -----------
Anub'arak  Alexstrasza  Azmodan    Abathur           Alarak
Artanis    Ana          Cassia     Chromie           Arthas
Diablo     Anduin       Falstad    Deathwing         Blaze
E.T.C.     Auriel       Gazlowe    Fenix             Chen
Garrosh    Brightwing   Greymane   Gall              Cho
Johanna    Deckard      Hanzo      Gul'dan           D.Va
Mei        Kharazim     Jaina      Junkrat           Dehaka
Muradin    Li Li        Kael'thas  Kel'Thuzad        Genji
Stitches   Lt. Morales  Li-Ming    Maiev             Hogger
Varian     Lúcio        Lunara     Medivh            Illidan
           Malfurion    Nazeebo    Mephisto          Imperius
           Rehgar       Nova       Murky             Kerrigan
           Stukov       Orphea     Probius           Leoric
           Tyrande      Raynor     Ragnaros          Mal'Ganis
           Uther        Sonya      Samuro          

In [11]:
hero1=stats[np.argwhere(heroes==['Anduin'])].flatten()
hero2=stats[np.argwhere(heroes==['Brightwing'])].flatten()
print(tabulate(np.transpose(np.stack((header,hero1,hero2)))))

----------  ---------  ---------
Games       257622     287005
Win %           50         52
Avg Length      19.17      19.07
T/D              5.8        5.1
Takedowns       13.4       13.1
Kills            1.4        1.4
Deaths           2.3        2.6
Hero Dmg     24212      15811
Siege Dmg    26810      31705
Healing      74049      86209
Self Heal     6423          0
Dmg Taken    32207      34818
XP            6752       7630
DPS             91.8      129.3
Range            5.5        1.3
HP            2074       3851
----------  ---------  ---------


In [112]:
indx = 5 #Data set index
numclass = 6
print('Dimensions =',data[indx][0])
print('Loss =',data[indx][1])
kmeans_classify(data[indx][2],exclusions,numclass)

#Dim:Comments
#2,3:healers in tank as before
#4: Unstable. Sonya back in tank. Worse than before
#5: Unstable. Sonya out. Class 0 derived from 5: lower damage, higher self-heal and tank
#6: Unstable. common outcome:
#tank and healers classes. good damage high kills. good siege high tank. good damage good tank high kill  

Dimensions = 6
Loss = 0.00027452537324279547
---------  ---------  ----------------  -----------  -----------  ---------
Anub'arak  Azmodan    Abathur           Arthas       Alexstrasza  Alarak
Artanis    Falstad    Chromie           Blaze        Ana          Cassia
Diablo     Jaina      Fenix             Chen         Anduin       D.Va
E.T.C.     Kael'thas  Gall              Cho          Auriel       Deathwing
Garrosh    Li-Ming    Junkrat           Dehaka       Brightwing   Gazlowe
Johanna    Lunara     Kel'Thuzad        Illidan      Deckard      Genji
Mei        Nazeebo    Mephisto          Imperius     Kharazim     Greymane
Muradin    Raynor     Murky             Kerrigan     Li Li        Gul'dan
Stitches   Sonya      Probius           Leoric       Lt. Morales  Hanzo
Varian     Sylvanas   Ragnaros          Mal'Ganis    Lúcio        Hogger
           Tassadar   Samuro            Rexxar       Malfurion    Maiev
           Tychus     Sgt. Hammer       The Butcher  Rehgar       Malthael

In [87]:
hero1=stats[np.argwhere(heroes==['Genji'])].flatten()
hero2=stats[np.argwhere(heroes==['Tracer'])].flatten()
hero3=stats[np.argwhere(heroes==['Yrel'])].flatten()
print(tabulate(np.transpose(np.stack((header,hero1,hero2,hero3)))))

----------  --------  --------  --------
Games       50479     57214     24737
Win %          44.1      53.8      46.4
Avg Length     18.94     18.87     19.11
T/D             3.8       4.2       4.1
Takedowns      13.7      14.1      10.9
Kills           5.7       6.1       2.3
Deaths          3.6       3.4       2.7
Hero Dmg    52320     52842     35899
Siege Dmg   46460     58983     81076
Healing         0      4502     10700
Self Heal    8241     12708     27078
Dmg Taken   51226     44230     77293
XP           9374     11141     13142
DPS           133.1     162.1     176.7
Range           5.5       2         1.5
HP           2111      3347      3966
----------  --------  --------  --------


In [114]:
#3 hidden layers
data = pickle.load(open('auto2_exp0.pickle','rb'))

In [232]:
indx = 20 #Data set index
numclasses = 5
print('Encoding dim1 =',data[indx][1])
print('Encoding dim2 =',data[indx][0])
print('Loss=',data[indx][2])
kmeans_classify(data[indx][3],exclusions,numclasses)


#Dim1:Dim2:Comments
#3:2:Inspect
#4:2:Too much mixing
#5:2: healer, tank mixed
#6,7:2:Inspect
#8:2:damage, tank mixed
#4:3:damage, tank mixed
#5:3:Sonya tank
#6,7,8:3:healers tank
#5,8:4:Inspect
#6:4:damage in tank
#7:4:healers in damage
#6:5:damage in tank
#7:5:healers in tank
#8:5:healers, damage in tank
#7:6:Very mixed. Inspect
#8:6:Inspect

Encoding dim1 = 8
Encoding dim2 = 6
Loss= 0.00023600272834300995
----------------  ---------  -----------  -----------  -----------
Arthas            Anub'arak  Alexstrasza  Alarak       Abathur
Chen              Artanis    Ana          Blaze        Azmodan
Cho               Diablo     Anduin       D.Va         Cassia
Illidan           E.T.C.     Auriel       Deathwing    Chromie
Kerrigan          Garrosh    Brightwing   Dehaka       Falstad
Mal'Ganis         Johanna    Deckard      Fenix        Gall
Rexxar            Mei        Kharazim     Gazlowe      Greymane
The Lost Vikings  Muradin    Li Li        Genji        Gul'dan
Tyrael            Sonya      Lt. Morales  Hogger       Hanzo
Yrel              Stitches   Lúcio        Imperius     Jaina
Zarya             Varian     Malfurion    Leoric       Junkrat
                             Rehgar       Maiev        Kael'thas
                             Stukov       Malthael     Kel'Thuzad
                             Tyrande      Medivh   

In [138]:
indx = 20 #Data set index
numclasses = 6
print('Encoding dim1 =',data[indx][1])
print('Encoding dim2 =',data[indx][0])
print('Loss=',data[indx][2])
kmeans_classify(data[indx][3],exclusions,numclasses)

#Dim1:Dim2:Comments
#3:2:Inspect
#4:2:healers split: more healing vs more damage
#5:2:Kharazim front line
#6,7:2:mixed class
#8:2:Li-Ming tank
#4:3:tank damage dealers mixed
#5:3:healer split, mixed class
#6,7,8:3:healers misplaced
#5:4:Mixed class
#6:4:healer split, damage w/ tank
#7:4:healers misplaced
#8:4:Inspect
#6:5:Inspect
#7:5:healers misplaced
#8:5:tank, damage mixed
#7:6:healers misplaced
#8:6:Lowest loss

Encoding dim1 = 8
Encoding dim2 = 6
Loss= 0.00023600272834300995
---------  ----------------  -----------  ---------  -----------  -----------
Anub'arak  Arthas            Alexstrasza  Abathur    Cassia       Alarak
Artanis    Chen              Ana          Azmodan    Fenix        Blaze
Diablo     Cho               Anduin       Chromie    Gazlowe      D.Va
E.T.C.     Illidan           Auriel       Falstad    Greymane     Deathwing
Garrosh    Kerrigan          Brightwing   Gall       Gul'dan      Dehaka
Johanna    Mal'Ganis         Deckard      Hanzo      Kel'Thuzad   Genji
Mei        Rexxar            Kharazim     Jaina      Mephisto     Hogger
Muradin    The Lost Vikings  Li Li        Junkrat    Murky        Imperius
Sonya      Tyrael            Lt. Morales  Kael'thas  Nova         Leoric
Stitches   Yrel              Lúcio        Li-Ming    Orphea       Maiev
Varian     Zarya             Malfurion    Lunara     Probius      Malthael
                             Rehgar       Nazeebo   

The results are definitely more versatile relative to direct clustering. Many do not make sense, but some do. This could be an indication that there are too many different and arbitrary ways to classify the heroes, in which case one moves to analysis: pick a classification result and see what you can learn about how a given hero is being played.

Of course one could do more experiments including:
- Different hyperparameters or architectures
- Aim for lower/higher loss with autoencoders
- Try self organizing maps
- Increase number of classes
- Try different clustering technique, perhaps one a ven diagram type that allows for overlap
- Redo autoencoder but without internal stats; more direct comparison with plain clustering

Let's try the last one really quick

In [157]:
indx = 20 #Data set index
numclasses = 8
print('Encoding dim1 =',data[indx][1])
print('Encoding dim2 =',data[indx][0])
print('Loss=',data[indx][2])
kmeans_classify(data[indx][3],exclusions,numclasses)

#7 classes: Sonya and Varian still in Anubaral class, Genji still with Alarak
#8: Same. Hanzo is with siege damage class which is weird.

Encoding dim1 = 8
Encoding dim2 = 6
Loss= 0.00023600272834300995
----------  -------  -----------  ----------------  ---------  ---------  -----------  -----------
Anduin      Abathur  Alexstrasza  Arthas            Anub'arak  Azmodan    Cassia       Alarak
Brightwing  Chromie  Ana          Chen              Artanis    Falstad    Fenix        Blaze
Li Li       Gall     Auriel       Cho               Diablo     Jaina      Gazlowe      D.Va
Lúcio       Hanzo    Deckard      Illidan           E.T.C.     Kael'thas  Greymane     Deathwing
Malfurion   Junkrat  Kharazim     Kerrigan          Garrosh    Li-Ming    Gul'dan      Dehaka
Stukov      Probius  Lt. Morales  Mal'Ganis         Johanna    Lunara     Kel'Thuzad   Genji
Tyrande     Zagara   Rehgar       Rexxar            Mei        Nazeebo    Mephisto     Hogger
                     Uther        The Lost Vikings  Muradin    Raynor     Murky        Imperius
                     Whitemane    Tyrael            Sonya      Sylvanas   Nova     

In [158]:
hero1=stats[np.argwhere(heroes==['Hanzo'])].flatten()
hero2=stats[np.argwhere(heroes==['Alarak'])].flatten()
hero3=stats[np.argwhere(heroes==['Blaze'])].flatten()
hero4=stats[np.argwhere(heroes==['Illidan'])].flatten()
print(tabulate(np.transpose(np.stack((header,hero1,hero2,hero3,hero4)))))

----------  --------  --------  --------  --------
Games       90098     56246     91362     60151
Win %          45.4      48.9      50.9      51.3
Avg Length     18.96     18.43     19.16     18.61
T/D             4.5       4.2       3.8       2.6
Takedowns      12.9      14.8      12.8      11.7
Kills           4.7       5.6       2.4       4.6
Deaths          2.9       3.5       3.3       4.5
Hero Dmg    62821     55572     40579     37944
Siege Dmg   69607     46003     71933     68214
Healing         0         0         0         0
Self Heal       0     11935     18163     40067
Dmg Taken   25833     53196     74227     67968
XP          10829      9769     10843     12200
DPS           200.2     259       234.2     212.9
Range           5.5       1.5       3.5       2
HP           2444      2718      5102      2613
----------  --------  --------  --------  --------


In [136]:
indx = 14 #Data set index
print('Encoding dim1 =',data[indx][1])
print('Encoding dim2 =',data[indx][0])
print('Loss=',data[indx][2])
kmeans_classify(data[indx][3],exclusions,5)

Encoding dim1 = 8
Encoding dim2 = 5
Loss= 0.00028589944122359157
----------  -----------  -----------  ---------  ----------------
Anduin      Alexstrasza  Alarak       Abathur    Arthas
Anub'arak   Ana          Chromie      Azmodan    Blaze
Artanis     Auriel       Deathwing    Cassia     Chen
Brightwing  Deckard      Fenix        Falstad    Cho
Diablo      Kharazim     Gall         Gazlowe    D.Va
E.T.C.      Li Li        Genji        Greymane   Dehaka
Garrosh     Lt. Morales  Kel'Thuzad   Gul'dan    Hogger
Johanna     Lúcio        Maiev        Hanzo      Illidan
Li-Ming     Malfurion    Medivh       Jaina      Imperius
Mei         Rehgar       Mephisto     Junkrat    Kerrigan
Muradin     Stukov       Nova         Kael'thas  Leoric
Stitches    Tyrande      Probius      Lunara     Mal'Ganis
Valla       Uther        Qhira        Nazeebo    Malthael
Varian      Whitemane    Samuro       Orphea     Murky
                         Sgt. Hammer  Ragnaros   Rexxar
                         Tra

In [147]:
losses = []
for i in range(len(data)-1):
    losses.append(data[i+1][2])

In [148]:
losses

[0.002310005482286215,
 0.002342197811231017,
 0.0017399645876139402,
 0.0017873409669846296,
 0.001989260083064437,
 0.0013815489364787936,
 0.0031491792760789394,
 0.0017219644505530596,
 0.0016804293263703585,
 0.0011243980843573809,
 0.000634494936093688,
 0.0017428058199584484,
 0.0014256698777899146,
 0.00037993403384462,
 0.0007330383523367345,
 0.0013214664068073034,
 0.00031921060872264206,
 0.00028589944122359157,
 0.0006675118347629905,
 0.00023600272834300995]

In [76]:
encoded_stats = encoder.predict(X)
kmeans_classify(encoded_stats,exclusions,6)

-----------  ---------  -----------  ---------  ----------------  -----------
Ana          Anub'arak  Alexstrasza  Abathur    Cho               Alarak
Anduin       Artanis    Deckard      Azmodan    D.Va              Arthas
Brightwing   Auriel     Diablo       Cassia     Deathwing         Blaze
Li Li        E.T.C.     Li-Ming      Falstad    Fenix             Chen
Lt. Morales  Garrosh    Muradin      Hanzo      Hogger            Chromie
Lúcio        Johanna    Rehgar       Jaina      Maiev             Dehaka
Malfurion    Kharazim   Stitches     Kael'thas  Malthael          Gall
Stukov       Raynor     Tyrande      Lunara     Mephisto          Gazlowe
             Tychus     Uther        Mei        Murky             Genji
             Varian     Valla        Nazeebo    Probius           Greymane
                        Whitemane    Nova       Ragnaros          Gul'dan
                                     Sonya      Rexxar            Illidan
                                     Sylvanas 

In [335]:
encoder.predict(X)[:10]

array([[0.        , 0.        , 0.        , 0.47974724],
       [0.        , 0.        , 0.        , 0.39854905],
       [0.        , 0.        , 0.        , 0.26497138],
       [0.        , 0.        , 0.        , 0.4320862 ],
       [0.        , 0.        , 0.        , 0.24292949],
       [0.        , 0.        , 0.        , 0.3536292 ],
       [0.        , 0.        , 0.        , 0.37702087],
       [0.        , 0.        , 0.        , 0.4182535 ],
       [0.        , 0.        , 0.        , 0.38674286],
       [0.        , 0.        , 0.        , 0.31781885]], dtype=float32)

In [362]:
autoencoder.fit(X,X,
                epochs = 1,
                batch_size=90,
                verbose='auto');

1/1 [==============================] - 0s 6ms/step - loss: 0.0045


In [40]:
X.shape

(90, 12)

In [242]:
input_dim = X.shape[1]
encoding_dim = 5
Xin = keras.Input(shape=(input_dim,))
encoded1 = layers.Dense(8,activation='relu')(Xin)
encoded2 = layers.Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = layers.Dense(8, activation='relu')(encoded2)
decoded2 = layers.Dense(input_dim, activation='relu')(decoded1)

autoencoder = keras.Model(Xin, decoded2)

encoder = keras.Model(Xin,encoded2)
#encoded_input = keras.Input(shape=(encoding_dim))
#decoder_layer = autoencoder.layers[-1]
#decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam',loss='mean_squared_error')

In [253]:
Xenc = encoder.predict(X)
kmeans_classify(Xenc,exclusions,5)

-----------  ---------  -----------  ---------  ----------------
Abathur      Anub'arak  Alexstrasza  Azmodan    Alarak
Chromie      Artanis    Ana          Cassia     Blaze
Fenix        Arthas     Anduin       Falstad    D.Va
Gall         Chen       Auriel       Gazlowe    Deathwing
Junkrat      Cho        Brightwing   Greymane   Dehaka
Kel'Thuzad   Diablo     Deckard      Gul'dan    Genji
Mephisto     E.T.C.     Kharazim     Hanzo      Hogger
Murky        Garrosh    Li Li        Jaina      Illidan
Probius      Imperius   Lt. Morales  Kael'thas  Kerrigan
Ragnaros     Johanna    Lúcio        Li-Ming    Leoric
Samuro       Mal'Ganis  Malfurion    Lunara     Maiev
Sgt. Hammer  Mei        Rehgar       Medivh     Malthael
Xul          Muradin    Stukov       Nazeebo    Qhira
Zagara       Stitches   Tyrande      Nova       Rexxar
             Tyrael     Uther        Orphea     Sonya
             Varian     Whitemane    Raynor     The Butcher
                                     Sylvanas   T

In [251]:
autoencoder.fit(X,X,epochs = 100,batch_size=90)

Epoch 1/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0012
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 0.0012
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0012
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 0.0012
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 0.0012
Epoch 6/100
1/1 [==============================] - 0s 21ms/step - loss: 0.0012
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 0.0012
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0012
Epoch 9/100
1/1 [==============================] - 0s 12ms/step - loss: 0.0012
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 0.0012
Epoch 11/100
1/1 [==============================] - 0s 12ms/step - loss: 0.0012
Epoch 12/100
1/1 [==============================] - 0s 26ms/step - loss: 0.0012
Epoch 13/100
1/1 [==============================] 

In [56]:
hero1=stats[np.argwhere(heroes==['Diablo'])].flatten()
hero2=stats[np.argwhere(heroes==['Anub\'arak'])].flatten()
print(tabulate(np.transpose(np.stack((header,hero1,hero2)))))

----------  ---------  --------
Games       201098     134101
Win %           47         52
Avg Length      19.09      19
T/D              3          3.8
Takedowns       13         12.2
Kills            2.7        2
Deaths           4.3        3.2
Hero Dmg     35175      33145
Siege Dmg    35505      46123
Healing          0          0
Self Heal    27004      23304
Dmg Taken   100042      68433
XP            8813       9382
DPS            153.8       88.7
Range            2.5        5.5
HP            4073       2517
----------  ---------  --------


In [85]:
Xenc[:10]

array([[0.        , 0.04488558],
       [0.        , 0.05266437],
       [0.        , 0.05504235],
       [0.        , 0.05574748],
       [0.        , 0.0389156 ],
       [0.        , 0.04844641],
       [0.        , 0.        ],
       [0.        , 0.06315924],
       [0.        , 0.01752857],
       [0.        , 0.06055853]], dtype=float32)

In [257]:
output = pickle.load(open('test1.pickle','rb'))

In [272]:
kmeans_classify(output[1][1],exclusions,6)

-----------  ---------  ---------  -----------  --------  ----------------
Chromie      Anub'arak  D.Va       Alexstrasza  Arthas    Abathur
Fenix        Artanis    Deathwing  Ana          Blaze     Alarak
Gall         Chen       Greymane   Anduin       Cho       Azmodan
Junkrat      Diablo     Gul'dan    Auriel       Dehaka    Cassia
Kel'Thuzad   E.T.C.     Hanzo      Brightwing   Illidan   Falstad
Maiev        Garrosh    Hogger     Deckard      Imperius  Gazlowe
Mephisto     Johanna    Malthael   Kharazim     Kerrigan  Genji
Murky        Mal'Ganis  Nova       Li Li        Leoric    Jaina
Probius      Mei        Orphea     Lt. Morales  Li-Ming   Kael'thas
Samuro       Muradin    Ragnaros   Lúcio        Raynor    Lunara
Sgt. Hammer  Stitches   Valeera    Malfurion    Sonya     Medivh
Zagara       Tyrael     Xul        Rehgar       Thrall    Nazeebo
             Varian     Zeratul    Stukov       Tychus    Qhira
                                   Tyrande      Valla     Rexxar
          

In [280]:
np.savez('norm_stats.npz',norm_stats=norm_stats)

In [281]:
norm_stats == np.load('norm_stats.npz')['norm_stats']

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])